In [1]:
%cd C:/Users/alapa/funiegan/funiegan_1.0

C:\Users\alapa\funiegan\funiegan_1.0


C:\Users\alapa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from piqa import PSNR
import torch
import cv2
import numpy as np
from skimage.measure import shannon_entropy


In [3]:

# entropy definition

def compute_entropy(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    
    # Compute entropy
    entropy_value = shannon_entropy(image)
    return entropy_value


In [5]:
# uiqm definitions


def compute_uicm(image):
    """
    Compute UICM (Underwater Image Colorfulness Measure).
    """
    r, g, b = cv2.split(image.astype(np.float32))
    rg = r - g
    yb = (r + g) / 2 - b
    rg_std = np.std(rg)
    yb_std = np.std(yb)
    uicm = np.sqrt(rg_std**2 + yb_std**2)
    return uicm

def compute_uism(image):
    """
    Compute UISM (Underwater Image Sharpness Measure).
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    uism = np.mean(np.abs(laplacian))
    return uism

def compute_uiconm(image):
    """
    Compute UIConM (Underwater Image Contrast Measure).
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist, _ = np.histogram(gray.ravel(), bins=256, range=(0, 256))
    prob = hist / hist.sum()
    prob = prob[prob > 0]  # Avoid log(0)
    uiconm = -np.sum(prob * np.log2(prob))  # Shannon entropy
    return uiconm

def compute_uiqm(image_path):
    """
    Compute UIQM (Underwater Image Quality Measure).
    Args:
        image_path (str): Path to the input image.
    Returns:
        float: UIQM value.
    """
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")

    # Compute components
    uicm = compute_uicm(image)
    uism = compute_uism(image)
    uiconm = compute_uiconm(image)

    # Compute UIQM
    uiqm = 0.0282 * uicm + 0.2953 * uism + 3.5753 * uiconm
    return uiqm


In [6]:
# ssim value function

from skimage.metrics import structural_similarity as ssim

def calculate_ssim(original_path, processed_path):
    """
    Calculate the SSIM score between an original image and a processed image.
    
    Args:
        original_path (str): Path to the original image.
        processed_path (str): Path to the processed image.

    Returns:
        float: SSIM score.
    """
    # Load images
    original = cv2.imread(original_path, cv2.IMREAD_COLOR)
    processed = cv2.imread(processed_path, cv2.IMREAD_COLOR)

    # Convert to grayscale (if needed for SSIM calculation)
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    processed_gray = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)

    # Compute SSIM
    ssim_score, _ = ssim(original_gray, processed_gray, full=True)
    return ssim_score


# only this below cell is for checking the best psnr model of each model

In [10]:
import os
import cv2
import numpy as np
import torch

# Paths to the directories
original_dir = "EUVP/test_samples/Inp"
processed_base_dir = "EUVP/test_samples/GTr"

# Loop through all images in the GTr folder
for original_file in os.listdir(original_dir):
    if original_file.endswith(".jpg"):  # Filter for images
        original_path = os.path.join(original_dir, original_file)
        original = cv2.imread(original_path)
        original = original.astype(np.float32) / 255.0
        
        best_value = 0
        best_model_number = 0
        
        # Loop through all generator models
        #for i in range(71, 76):
        processed_path = os.path.join(processed_base_dir, original_file)
        if os.path.exists(processed_path):  # Ensure the file exists
            processed = cv2.imread(processed_path)
            processed = processed.astype(np.float32) / 255.0

            original_tensor = torch.from_numpy(original).permute(2, 0, 1).unsqueeze(0)  # [Batch, Channels, H, W]
            processed_tensor = torch.from_numpy(processed).permute(2, 0, 1).unsqueeze(0)  # [Batch, Channels, H, W]

            try:
                entropy_value = compute_entropy(processed_path) # entropy value calulation
                uiqm_value = compute_uiqm(processed_path)
                ssim_score = calculate_ssim(original_path, processed_path)
            except FileNotFoundError as e:
                print(e)
            psnr = PSNR()
            psnr_value = psnr(processed_tensor, original_tensor)

            if psnr_value.item() > best_value:
                best_value = psnr_value.item()
                #best_model_number = i

        print(f"Image: {original_file},    Best_PSNR: {best_value:.3f} dB,    SSIM: {ssim_score:.3f},    Model_Number: {best_model_number},    Entropy: {entropy_value:.3f},    UIQM: {uiqm_value:.3f}")


Image: test_p0_.jpg,    Best_PSNR: 20.685 dB,    SSIM: 0.885,    Model_Number: 0,    Entropy: 7.101,    UIQM: 30.164
Image: test_p100_.jpg,    Best_PSNR: 20.881 dB,    SSIM: 0.892,    Model_Number: 0,    Entropy: 7.792,    UIQM: 33.282
Image: test_p101_.jpg,    Best_PSNR: 21.199 dB,    SSIM: 0.893,    Model_Number: 0,    Entropy: 7.311,    UIQM: 30.543
Image: test_p102_.jpg,    Best_PSNR: 16.156 dB,    SSIM: 0.879,    Model_Number: 0,    Entropy: 6.480,    UIQM: 28.850
Image: test_p104_.jpg,    Best_PSNR: 22.623 dB,    SSIM: 0.919,    Model_Number: 0,    Entropy: 7.239,    UIQM: 32.370
Image: test_p105_.jpg,    Best_PSNR: 20.221 dB,    SSIM: 0.823,    Model_Number: 0,    Entropy: 7.392,    UIQM: 36.004
Image: test_p106_.jpg,    Best_PSNR: 20.273 dB,    SSIM: 0.825,    Model_Number: 0,    Entropy: 7.432,    UIQM: 40.021
Image: test_p107_.jpg,    Best_PSNR: 17.416 dB,    SSIM: 0.908,    Model_Number: 0,    Entropy: 7.294,    UIQM: 32.220
Image: test_p109_.jpg,    Best_PSNR: 24.283 dB,   

KeyboardInterrupt: 

In [ ]:
import os
import cv2
import numpy as np
import torch

# Paths to the directories
original_dir = "EUVP/test_samples/Inp"
processed_base_dir = "EUVP/test_samples/Inp"

# Loop through all images in the GTr folder
for original_file in os.listdir(original_dir):
    if original_file.endswith(".jpg"):  # Filter for images
        original_path = os.path.join(original_dir, original_file)
        original = cv2.imread(original_path)
        original = original.astype(np.float32) / 255.0
        
        best_value = 0
        best_model_number = 0
        
        # Loop through all generator models
        for i in range(71, 76):
            processed_path = os.path.join(processed_base_dir, f'generator_{i}', original_file)
            if os.path.exists(processed_path):  # Ensure the file exists
                processed = cv2.imread(processed_path)
                processed = processed.astype(np.float32) / 255.0

                original_tensor = torch.from_numpy(original).permute(2, 0, 1).unsqueeze(0)  # [Batch, Channels, H, W]
                processed_tensor = torch.from_numpy(processed).permute(2, 0, 1).unsqueeze(0)  # [Batch, Channels, H, W]

                try:
                    entropy_value = compute_entropy(processed_path) # entropy value calulation
                    uiqm_value = compute_uiqm(processed_path)
                    ssim_score = calculate_ssim(original_path, processed_path)
                except FileNotFoundError as e:
                    print(e)
                psnr = PSNR()
                psnr_value = psnr(processed_tensor, original_tensor)

                if psnr_value.item() > best_value:
                    best_value = psnr_value.item()
                    best_model_number = i

        print(f"Image: {original_file},    Best_PSNR: {best_value:.3f} dB,    SSIM: {ssim_score:.3f},    Model_Number: {best_model_number},    Entropy: {entropy_value:.3f},    UIQM: {uiqm_value:.3f}")


In [ ]:
import csv

def parse_line_to_dict(line):
    # Split the line by commas to separate key-value pairs
    key_value_pairs = line.strip().split(',')
    data_dict = {}
    
    # Iterate over each key-value pair and split by colon
    for pair in key_value_pairs:
        key, value = pair.split(':', 1)  # Split only on the first colon
        data_dict[key.strip()] = value.strip()
    
    return data_dict

def process_file(input_file, output_file):
    # Open the input text file and output CSV file
    with open(input_file, 'r') as file:
        lines = file.readlines()
    
    # Parse the lines into dictionaries
    data_list = [parse_line_to_dict(line) for line in lines]
    
    # Get all column headings (keys) from the first entry in the list
    columns = data_list[0].keys()
    
    # Write the data to a CSV file
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writeheader()
        writer.writerows(data_list)
    print(f"Data has been written to {output_file}")

# Example usage
input_file = 'E:/research/input.txt'  # Your input text file
output_file = 'E:/research/output.csv'  # The output CSV file

process_file(input_file, output_file)


# psnr and ssim new metrics


In [ ]:

## python libs
import numpy as np
from PIL import Image
from glob import glob
from os.path import join
from ntpath import basename
## local libs


from __future__ import division
import numpy as np
import math
from scipy.ndimage import gaussian_filter



def getSSIM(X, Y):
    """
       Computes the mean structural similarity between two images.
    """
    assert (X.shape == Y.shape), "Image-patche provided have different dimensions"
    nch = 1 if X.ndim==2 else X.shape[-1]
    mssim = []
    for ch in range(nch):
        Xc, Yc = X[...,ch].astype(np.float64), Y[...,ch].astype(np.float64)
        mssim.append(compute_ssim(Xc, Yc))
    return np.mean(mssim)


def compute_ssim(X, Y):
    """
       Compute the structural similarity per single channel (given two images)
    """
    # variables are initialized as suggested in the paper
    K1 = 0.01
    K2 = 0.03
    sigma = 1.5
    win_size = 5   

    # means
    ux = gaussian_filter(X, sigma)
    uy = gaussian_filter(Y, sigma)

    # variances and covariances
    uxx = gaussian_filter(X * X, sigma)
    uyy = gaussian_filter(Y * Y, sigma)
    uxy = gaussian_filter(X * Y, sigma)

    # normalize by unbiased estimate of std dev 
    N = win_size ** X.ndim
    unbiased_norm = N / (N - 1)  # eq. 4 of the paper
    vx  = (uxx - ux * ux) * unbiased_norm
    vy  = (uyy - uy * uy) * unbiased_norm
    vxy = (uxy - ux * uy) * unbiased_norm

    R = 255
    C1 = (K1 * R) ** 2
    C2 = (K2 * R) ** 2
    # compute SSIM (eq. 13 of the paper)
    sim = (2 * ux * uy + C1) * (2 * vxy + C2)
    D = (ux ** 2 + uy ** 2 + C1) * (vx + vy + C2)
    SSIM = sim/D 
    mssim = SSIM.mean()

    return mssim



def getPSNR(X, Y):
    #assume RGB image
    target_data = np.array(X, dtype=np.float64)
    ref_data = np.array(Y, dtype=np.float64)
    diff = ref_data - target_data
    diff = diff.flatten('C')
    rmse = math.sqrt(np.mean(diff ** 2.) )
    if rmse == 0: return 100
    else: return 20*math.log10(255.0/rmse)




## compares avg ssim and psnr 
def SSIMs_PSNRs(gtr_dir, gen_dir, im_res=(256, 256)):
    """
        - gtr_dir contain ground-truths
        - gen_dir contain generated images 
    """
    gtr_paths = sorted(glob(join(gtr_dir, "*.*")))
    gen_paths = sorted(glob(join(gen_dir, "*.*")))
    ssims, psnrs = [], []

    for gtr_path, gen_path in zip(gtr_paths, gen_paths):
        gtr_f = basename(gtr_path).split('.')[0]
        gen_f = basename(gen_path).split('.')[0]
        if (gtr_f==gen_f):
            # assumes same filenames
            r_im = Image.open(gtr_path).resize(im_res)
            g_im = Image.open(gen_path).resize(im_res)
            # get ssim on RGB channels
            ssim = getSSIM(np.array(r_im), np.array(g_im))
            ssims.append(ssim)
            # get psnt on L channel (SOTA norm)
            r_im = r_im.convert("L"); g_im = g_im.convert("L")
            psnr = getPSNR(np.array(r_im), np.array(g_im))
            psnrs.append(psnr)
    return np.array(ssims), np.array(psnrs)



# gtr_dir = "/home/mbzirc/Downloads/AhsanBB/Dehazing/UEIB_Data/UEIB_Dataset/Data2/test/testB"

#gtr_dir = "E:/research/AI-image-upscaling-using-GAN-main/upscalar(ESRGAN)/results/E_Inp"
gtr_dir = "EUVP/test_samples/Inp"
#gtr_dir = "/home/xahid/datasets/released/UFO-120/TEST/hr/"
for i in range(71,76):
    #gen_path = (f"E:/research/AI-image-upscaling-using-GAN-main/upscalar(ESRGAN)/results/E_GTr")
    gen_path = (f"EUVP/test_samples/result/NLSelf/generator_{i}")
    #gen_dir = "EUVP/test_samples/result/Self/generator_"+str(i) 
    #gen_dir = "eval_data/ufo_test/deep-sesr/" 
    gen_dir = gen_path

    ### compute SSIM and PSNR
    SSIM_measures, PSNR_measures = SSIMs_PSNRs(gtr_dir, gen_dir)
    '''print ("SSIM on {0} samples".format(len(SSIM_measures)))
    print ("Mean: {0} std: {1}".format(np.mean(SSIM_measures), np.std(SSIM_measures)))

    print ("PSNR on {0} samples".format(len(PSNR_measures)))
    print ("Mean: {0} std: {1}".format(np.mean(PSNR_measures), np.std(PSNR_measures)))
    '''
    print("genarator:  {0} | Samples: {1} | SSIM Mean: {2:.4f} std: {3:.4f} | PSNR Mean: {5:.4f} std: {6:.4f}".format(i,len(SSIM_measures), np.mean(SSIM_measures), np.std(SSIM_measures), len(PSNR_measures), np.mean(PSNR_measures), np.std(PSNR_measures)))
    #print("| Samples: {0} | SSIM Mean: {1:.4f} std: {2:.4f} | PSNR Mean: {4:.4f} std: {5:.4f}".format(len(SSIM_measures), np.mean(SSIM_measures), np.std(SSIM_measures), len(PSNR_measures), np.mean(PSNR_measures), np.std(PSNR_measures)))




# previous Test files

In [ ]:
# Load images (ensure they have the same dimensions)
#original = cv2.imread('E:/research/funiegan_1.0/EUVP/test_samples/Inp/test_p37_.jpg')
#original = cv2.imread('E:/research/funiegan_1.0/EUVP/test_samples/result/Self/generator_23/test_p37_.jpg')  # Model output
original = cv2.imread("E:/research/funiegan_1.0/EUVP/test_samples/GTr/test_p37_.jpg")
processed = cv2.imread('E:/research/funiegan_1.0/EUVP/test_samples/result/Self/generator_2/test_p37_.jpg')   # Model output
#processed = cv2.imread('E:/research/funiegan_1.0/EUVP/test_samples/Inp/test_p37_.jpg')  # Model output
# processed = cv2.imread('EUVP/test_samples/result/metricsevaluation/SA(batch3)/underwater_dark/generator_3/264286_00007889.jpg')  # Ground truth
# processed = cv2.imread('EUVP/test_samples/result/metricsevaluation/noattention/underwater_dark/generator_8/264286_00007889.jpg') # Ground truth

# Convert to float32 and normalize to [0, 1]
original = original.astype(np.float32) / 255.0
processed = processed.astype(np.float32) / 255.0

# Convert to PyTorch tensors (add batch and channel dimensions)
original_tensor = torch.from_numpy(original).permute(2, 0, 1).unsqueeze(0)  # [Batch, Channels, H, W]
processed_tensor = torch.from_numpy(processed).permute(2, 0, 1).unsqueeze(0)  # [Batch, Channels, H, W]

# Compute PSNR
psnr = PSNR()
psnr_value = psnr(processed_tensor, original_tensor)
print(f"PSNR: {psnr_value.item()} dB")

In [ ]:
from skimage.measure import shannon_entropy
import cv2
import numpy as np

def compute_entropy(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    
    # Compute entropy
    entropy_value = shannon_entropy(image)
    return entropy_value

# Example usage
image_path = "C:/Users/alapa/funiegan/EUVP/test_samples/result/metricsevaluation/SA(batch8)/underwater_dark/generator_19/264286_00007889.jpg"
# image_path = "C:/Users/alapa/funiegan/EUVP/Paired/underwater_dark/trainB/264286_00007889.jpg"
# image_path = "C:/Users/alapa/imageupscaler/upscalar(ESRGAN)/results/RealESRGAN_x4.pth/output_264286_00007889.jpg"  # Replace with your image path
try:
    entropy_value = compute_entropy(image_path)
    print(f"Entropy of the image: {entropy_value}")
except FileNotFoundError as e:
    print(e)

In [ ]:
import cv2
import numpy as np

def compute_uicm(image):
    """
    Compute UICM (Underwater Image Colorfulness Measure).
    """
    r, g, b = cv2.split(image.astype(np.float32))
    rg = r - g
    yb = (r + g) / 2 - b
    rg_std = np.std(rg)
    yb_std = np.std(yb)
    uicm = np.sqrt(rg_std**2 + yb_std**2)
    return uicm

def compute_uism(image):
    """
    Compute UISM (Underwater Image Sharpness Measure).
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    uism = np.mean(np.abs(laplacian))
    return uism

def compute_uiconm(image):
    """
    Compute UIConM (Underwater Image Contrast Measure).
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist, _ = np.histogram(gray.ravel(), bins=256, range=(0, 256))
    prob = hist / hist.sum()
    prob = prob[prob > 0]  # Avoid log(0)
    uiconm = -np.sum(prob * np.log2(prob))  # Shannon entropy
    return uiconm

def compute_uiqm(image_path):
    """
    Compute UIQM (Underwater Image Quality Measure).
    Args:
        image_path (str): Path to the input image.
    Returns:
        float: UIQM value.
    """
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")

    # Compute components
    uicm = compute_uicm(image)
    uism = compute_uism(image)
    uiconm = compute_uiconm(image)

    # Compute UIQM
    uiqm = 0.0282 * uicm + 0.2953 * uism + 3.5753 * uiconm
    return uiqm

# Example usage

# image_path = "C:/Users/alapa/funiegan/EUVP/Paired/underwater_dark/trainA/264286_00007889.jpg"  
# image_path = "C:/Users/alapa/funiegan/EUVP/test_samples/result/metricsevaluation/SA(batch8)/underwater_dark/generator_19/264286_00007889.jpg"
# image_path = "C:/Users/alapa/funiegan/EUVP/Paired/underwater_dark/trainB/264286_00007889.jpg"
# image_path = "C:/Users/alapa/imageupscaler/upscalar(ESRGAN)/results/RealESRGAN_x8.pth/output_264286_00007889.jpg" 
try:
    uiqm_value = compute_uiqm(image_path)
    print(f"UIQM: {uiqm_value}")
except FileNotFoundError as e:
    print(e)